In [1]:
import os
import csv
import pickle
from datetime import datetime

def load_table(*files):
    t = []
    headers = None
    for f in files:
        fdet = os.path.splitext(f)[1]
        temp_t = []

        if fdet == '.csv':
            with open(f, 'r', encoding='utf-8') as file:
                read = csv.reader(file)
                head = next(read, None)

                if headers is None:
                    headers = head
                temp_t = [dict(zip(head, r)) for r in read]

        elif fdet == '.pkl':
            with open(f, 'rb') as file:
                temp_t = pickle.load(file)

                if headers is None:
                    headers = list(temp_t[0].keys())

        t.extend(temp_t)

    return t

def save_table(t, f, max_rows=None):
    fdet = os.path.splitext(f)[1]

    if max_rows is None:
        if fdet == '.csv':
            with open(f, 'w', encoding='utf-8') as file:
                if t:
                    w = csv.DictWriter(file, fieldnames=t[0].keys())
                    w.writeheader()
                    w.writerows(t)

        elif fdet == '.pkl':
            with open(f, 'wb') as file:
                pickle.dump(t, file)

        elif fdet == '.txt':
            with open(f, 'w', encoding='utf-8') as file:
                for row in t:
                    file.write('\t'.join(map(str, row.values())) + '\n')

    else:
        base, ext = os.path.splitext(f)
        for i in range(0, len(t), max_rows):
            part_file = f"{base}_part{i // max_rows + 1}{ext}"
            save_table(t[i:i + max_rows], part_file)

def get_rows_by_number(t, start, stop=None, copy_table=False):
    if stop is None:
        stop = start + 1
    rows = t[start:stop]

    if copy_table:
        return [r.copy() for r in rows]

    return rows

def get_rows_by_index(t, *vals, copy_table=False):
    rows = []
    for row in t:
        if row[list(row.keys())[0]] in vals:
            rows.append(row)

    if copy_table:
        return [row.copy() for row in rows]

    return rows

def get_column_types(t, by_number=True):
    column_types = {}

    if by_number:
        for i in range(len(t[0])):
            value = list(t[0].values())[i]
            col_type = str(type(value)).split("'")[1]
            column_types[i] = col_type

    else:
        for col_name in t[0].keys():
            value = t[0][col_name]
            col_type = str(type(value)).split("'")[1]
            column_types[col_name] = col_type

    return column_types

def set_column_types(t, types_dict, by_number=True):
    for row in t:
        for col, col_type in types_dict.items():
            if by_number:
                key = list(row.keys())[col]
            else:
                key = col

            value = row[key]

            if col_type == 'int':
                row[key] = int(value) if value is not None else None
            elif col_type == 'float':
                row[key] = float(value) if value is not None else None
            elif col_type == 'bool':
                row[key] = bool(value) if value is not None else None
            elif col_type == 'str':
                row[key] = str(value) if value is not None else None
            elif col_type == 'datetime':
                row[key] = datetime.fromisoformat(value) if value else None

def get_values(t, column=0):
    if isinstance(column, int):
        key = list(t[0].keys())[column]
    else:
        key = column

    return [row[key] for row in t]

def get_value(t, column=0):
    if len(t) != 1:
        return None

    if isinstance(column, int):
        key = list(t[0].keys())[column]
    else:
        key = column

    return t[0][key]

def set_values(t, values, column=0):
    if isinstance(column, int):
        key = list(t[0].keys())[column]
    else:
        key = column

    for row, value in zip(t, values):
        row[key] = value

def set_value(t, value, column=0):
    if len(t) != 1:
        return

    if isinstance(column, int):
        key = list(t[0].keys())[column]
    else:
        key = column

    t[0][key] = value

def print_table(t):
    if not t:
        print("Пустая таблица")
        return

    headers = t[0].keys()
    print("\t".join(headers))
    for row in t:
        print("\t".join(map(str, row.values())))
